In [ ]:
# Import libraries (you don't need to re-import libraries)
import glob
import numpy as np
import xarray as xr
import cartopy.crs as crs
import netCDF4 as nc
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
from cartopy.feature import NaturalEarthFeature
from wrf import (to_np, cartopy_xlim, cartopy_ylim,
                 getvar, latlon_coords, get_cartopy)

In [ ]:
# Load the WRFOUT file 
wrf_file_d2 = xr.open_dataset("/path/to/WRF/file")
# Load the WRFOUT file 
wrf_file_d3 = xr.open_dataset("/path/to/WRF/file")

#provide path to the file and opening it
file = "/path/to/WRF/file"
df=nc.Dataset(file)
# Provide the path to the directory containing the IMERG files
data_dir = "/path/to/IMERG/files/*.nc4"

# Provide path to the file and open it
file1 = "/path/to/WRF/file"
df1 = nc.Dataset(file1)

# Provide path to the MERRA observation file and open it
file2 = "/path/to/MERRA/file/file.nc"
ds2= xr.open_dataset(file2)

# Subplots for Terrain Height, SFCP and wind speed

In [ ]:
# Create a subplot with 1 row and 2 columns
fig, axs = plt.subplots(1, 2, figsize=(15, 6), subplot_kw={'projection': crs.PlateCarree()})

# Plot terrain height for Domain 
terrain_height_d2 = wrf_file_d2['HGT'].squeeze()
lon_d2 = wrf_file_d2['XLONG'].squeeze()
lat_d2 = wrf_file_d2['XLAT'].squeeze()
axs[0].coastlines('10m', linewidth=0.8)
shape_feature_d2 = ShapelyFeature(Reader('/path/to/the/shape/file/file.shp').geometries(),
                                    crs.PlateCarree(), edgecolor='black', facecolor='none')
axs[0].add_feature(shape_feature_d2)
im_d2 = axs[0].imshow(terrain_height_d2, cmap='terrain', extent=[lon_d2.min(), lon_d2.max(), lat_d2.min(), lat_d2.max()], origin='lower')
cbar_d2 = plt.colorbar(im_d2, ax=axs[0], extend='both')
cbar_d2.set_label("Terrain Height (m)")#, fontsize=20
axs[0].set_xlabel("Longitude")
axs[0].set_ylabel("Latitude")
axs[0].set_title("Terrain Height")#, fontsize=20
gl_d2 = axs[0].gridlines(draw_labels=True, color='grey', linestyle='dotted')
gl_d2.top_labels = False
gl_d2.right_labels = False

# Plot MSLP and Wind Vector for Domain 
psfc_hpa = wrf_file_d3['PSFC'][0, :, :] * 0.01
vertical_level = 12
u_wind_d3 = wrf_file_d3['U'][:, vertical_level, :, :]
v_wind_d3 = wrf_file_d3['V'][:, vertical_level, :, :]
lats_d3 = wrf_file_d3['XLAT'][0, :, :]
lons_d3 = wrf_file_d3['XLONG'][0, :, :]
contour = axs[1].contourf(lons_d3, lats_d3, psfc_hpa, levels=np.arange(900, 1050, 10), cmap='YlGnBu', extend='both')
axs[1].coastlines()
shape_feature_d3 = ShapelyFeature(Reader('/path/to/the/shape/file/file.shp').geometries(),
                                    crs.PlateCarree(), edgecolor='black', facecolor='none')
axs[1].add_feature(shape_feature_d3)
cbar_d3 = plt.colorbar(contour, ax=axs[1], orientation='vertical', pad=0.05, extend = 'both')#, extendrect=True
cbar_d3.set_label('Mean Sea Level Pressure (hPa)')
density = 10 
arrow_positions_lons = lons_d3[::density, ::density]
arrow_positions_lats = lats_d3[::density, ::density]
u_wind_subset = u_wind_d3[0, ::density, ::density]
v_wind_subset = v_wind_d3[0, ::density, ::density]

axs[1].quiver(arrow_positions_lons, arrow_positions_lats, u_wind_subset, v_wind_subset,
              pivot='middle', color='red', scale=200)
axs[1].set_title('MSLP and Wind Vectors')
axs[1].set_xlabel('Longitude')
axs[1].set_ylabel('Latitude')
gl_d3 = axs[1].gridlines(draw_labels=True, color='grey', linestyle='dotted')
gl_d3.top_labels = False
gl_d3.right_labels = False

# Show the subplot
plt.show()

# Subplots for Precipitation

In [ ]:
#Import variables
rainc = getvar(df,'RAINC')
rainsh = getvar(df,'RAINSH')
rainnc = getvar(df,'RAINNC')
prec=rainc+rainsh+rainnc
lats,lons = latlon_coords(rainnc)
# Your code for the first subplot
#fig = plt.figure(figsize=(10, 6))
fig, axs = plt.subplots(1, 2, figsize=(15, 6), subplot_kw={'projection': crs.PlateCarree()})#

# Subplot 1
ax1 = axs[0]
ax1 = plt.subplot(121, projection=crs.PlateCarree())
prec_c = ax1.contourf(lons, lats, prec, levels=np.arange(5, 450, 5),
                      transform=crs.PlateCarree(), cmap='magma_r', extend='both')
cbar = plt.colorbar(prec_c, ax=ax1, shrink=0.9)
cbar.set_label('precipitation (mm)', fontsize=12)
cbar.ax.tick_params(labelsize=8)
shape_feature0 = ShapelyFeature(Reader('/path/to/the/shape/file/file.shp').geometries(),
                                    crs.PlateCarree(), edgecolor='black', facecolor='none')

ax1.add_feature(shape_feature0)
ax1.coastlines('10m', linewidth=0.8)
ax1.set_title("Accumulated precipitation (SIMULATED)", fontsize=12)
gl = ax1.gridlines(crs=crs.PlateCarree(), draw_labels=True, color='grey', linestyle='dotted')
gl.top_labels = False
gl.right_labels = False


# Initialize the accumulated precipitation variable
acc_prcp = np.zeros((20, 15))

# Iterate over the files
for file in glob.glob(data_dir):
    # Open the file
    ds = xr.open_dataset(file)
    # Get the precipitation data
    prcp = ds.precipitationCal
    prcp = prcp.mean(dim='time')
    # Add the precipitation to the accumulated precipitation variable
    acc_prcp += prcp

# Additional code for the second subplot
lat = acc_prcp.lat.data
lon = acc_prcp.lon.data
Prcp = acc_prcp.data
Prcp = np.nan_to_num(Prcp)
lats1, lons1 = np.meshgrid(lat, lon)

# subplot 2
ax2 = axs[1]
ax2 = plt.subplot(122, projection=crs.PlateCarree())
# Add colorbar
cmap = get_cmap('magma_r')
#cax = fig2.add_axes([0.95, 0.15, 0.03, 0.7])
Prec_c = ax2.contourf(lons1, lats1, Prcp, levels=np.arange(5, 450, 5),
                               transform=crs.PlateCarree(), cmap=cmap, extend='both')#, cax=cax
cb = plt.colorbar(Prec_c, ax=ax2, shrink=0.9)
cb.set_label('Precipitation (mm)', fontsize=12)#
cb.ax.tick_params(labelsize=8)
shape_feature1 = ShapelyFeature(Reader('/path/to/the/shape/file/file.shp').geometries(),
                                    crs.PlateCarree(), edgecolor='black', facecolor='none')
ax2.add_feature(shape_feature1)
ax2.coastlines('10m', linewidth=0.8)

# Add title
ax2.set_title('Accumulated precipitation (IMERG)', fontsize=12)
ax2.set_xlabel('Lat', fontsize=12)
ax2.set_ylabel('Lon', fontsize=12)
gl = ax2.gridlines(crs=crs.PlateCarree(), color="grey", linestyle="dotted", draw_labels=True)
gl.top_labels = False
gl.right_labels = False

# Display the subplots
plt.show()


# Cloud fractiona and cloud albedo

In [ ]:
# Extract variables
cldfra_cfs = getvar(df1,'CLDFRA')
lats_cfs,lons_cfs = latlon_coords(cldfra_cfs)
cldabd_cas = getvar(df1,'ALBBCK')
lats_cas,lons_cas = latlon_coords(cldabd_cas)

cldfra = ds2.ISCCPCLDFRC
clfra_slice = cldfra.sel(time = 'YYYY-MM-DDTHH:mm:00.000000000')
lat   = clfra_slice.lat.data
lon   = clfra_slice.lon.data
clfra = clfra_slice.data
clfra_cfo = np.nan_to_num(clfra)
lats_cfo, lons_cfo = np.meshgrid(lat, lon)

cldabd = ds2.ISCCPALB
clabd_slice = cldabd.sel(time='YYYY-MM-DDTHH:mm:00.000000000')
lat1   = clabd_slice.lat.data
lon1   = clabd_slice.lon.data
clabd = clabd_slice.data
clabd_clo = np.nan_to_num(clabd)
lats_cao, lons_cao = np.meshgrid(lat1, lon1)

cldabd = ds2.ISCCPALB
clabd_slice = cldabd.sel(time='YYYY-MM-DDTHH:mm:00.000000000')
latx   = clabd_slice.lat.data
lonx   = clabd_slice.lon.data
clabd = clabd_slice.data
clabd_cao = np.nan_to_num(clabd)
lats_cao, lons_cao = np.meshgrid(latx, lonx)

# Create a figure with 1 row and 2 columns
fig, axs = plt.subplots(2, 2, figsize=(15, 15), subplot_kw={'projection': crs.PlateCarree()})

# Plot the first subplot
ax1 = axs[0,0]
clf_sim = ax1.contourf(lons_cfs, lats_cfs, cldfra_cfs[25,:,:,], levels=np.arange(0.3,1,0.01),
                      transform=crs.PlateCarree(), cmap='Reds', extend='both')
shape_feature1 = ShapelyFeature(Reader('/path/to/the/shape/file/file.shp').geometries(),
                                crs.PlateCarree(), edgecolor='black', facecolor='none')
ax1.add_feature(shape_feature1)
ax1.coastlines('10m', linewidth=0.8)
cbar1 = plt.colorbar(clf_sim, ax=ax1, shrink=0.9)
cbar1.set_label('Cloud fraction', fontsize=12)
cbar1.ax.tick_params(labelsize=8)
ax1.set_title("Cloud Fraction (SIMULATED)", fontsize=12)
gl1 = ax1.gridlines(crs=crs.PlateCarree(), draw_labels=True, color='grey', linestyle='dotted')
gl1.top_labels = False
gl1.right_labels = False

# Plot the second subplot
ax2 = axs[0,1]
cmap = get_cmap('Reds')
#cax2 = fig.add_axes([0.95, 0.15, 0.03, 0.7])
clf_obs = ax2.contourf(lons_cfo, lats_cfo, clfra_cfo, levels=np.arange(0.3, 1, 0.01),
                       transform=crs.PlateCarree(), cmap=cmap, extend='both')
shape_feature2 = ShapelyFeature(Reader('/path/to/the/shape/file/file.shp').geometries(),
                                crs.PlateCarree(), edgecolor='black', facecolor='none')
ax2.add_feature(shape_feature2)
ax2.coastlines('10m', linewidth=0.8)
cb2 = plt.colorbar(clf_obs, ax=ax2, shrink=0.8)
cb2.set_label('Cloud Fraction', fontsize=12)
cb2.ax.tick_params(labelsize=8)
ax2.set_title('Cloud fraction (MERRA)', fontsize=12)
gl2 = ax2.gridlines(crs=crs.PlateCarree(), draw_labels=True, color='grey', linestyle='dotted')
gl2.top_labels = False
gl2.right_labels = False

####################plotting the results####################### 
ax3 = axs[1,0]
cmap = get_cmap('jet')
cldabd_sim=ax3.contourf(lons_cas,lats_cas,cldabd_cas[:,:],levels=np.arange(0,1,0.01),
                   transform=crs.PlateCarree(),cmap=cmap,extend = 'both')

shape_feature3 = ShapelyFeature(Reader('/path/to/the/shape/file/file.shp').geometries(),
                                crs.PlateCarree(), edgecolor='black', facecolor='none')
ax3.add_feature(shape_feature3)
ax3.coastlines('10m', linewidth=0.8)
cbar3 = plt.colorbar(cldabd_sim, ax=ax3, shrink=0.9)
cbar3.set_label('Cloud Albedo',fontsize=12)
cbar3.ax.tick_params(labelsize=8)
ax3.set_title("Cloud albedo (SIMULATED)",fontsize=10)
gl3 = ax3.gridlines(crs=crs.PlateCarree(), draw_labels=True, color='grey', linestyle='dotted')
gl3.top_labels = False
gl3.right_labels = False

# Plot the 4th subplot
ax4 = axs[1,1]
cmap = get_cmap('jet')
cla_obs = ax4.contourf(lons_cao, lats_cao, clabd_cao, levels=np.arange(0, 1.0, 0.01),
                       transform=crs.PlateCarree(), cmap=cmap, extend='both')
shape_feature4 = ShapelyFeature(Reader('/path/to/the/shape/file/file.shp').geometries(),
                                crs.PlateCarree(), edgecolor='black', facecolor='none')
ax4.add_feature(shape_feature4)
ax4.coastlines('10m', linewidth=0.8)
cb4 = plt.colorbar(cla_obs, ax=ax4, shrink=0.8)
cb4.set_label('Cloud Albedo', fontsize=12)
cb4.ax.tick_params(labelsize=8)
ax4.set_title('Cloud Albedo (MERRA)', fontsize=12)
gl4 = ax4.gridlines(crs=crs.PlateCarree(), draw_labels=True, color='grey', linestyle='dotted')
gl4.top_labels = False
gl4.right_labels = False
# Display the subplots
#plt.tight_layout()
plt.show()


# Time series of precipitation observation vs simulation

In [ ]:
# Provide the path to the directory containing the IMERG files
data_dir1 = "/path/to/nc/file/*.nc4"
sim_hourly_rainfall = [0]
T = 0
for i in range (0, 23, 1):
    TTn = "%02d" % T
    string_n = '/path/to/wrf/file/wrfout_d03_YYYY-MM-DD_' + TTn + ':00:00'
    file_n = nc.Dataset(string_n)

    TTnp1 = "%02d" % (T+1)
    string_np1 = '/path/to/wrf/file/wrfout_d03_YYYY-MM-DD_' + TTnp1 + ':00:00'
    file_np1 = nc.Dataset(string_np1)

    rainc_n = getvar(file_n,'RAINC')
    rainsh_n = getvar(file_n,'RAINSH')
    rainnc_n = getvar(file_n,'RAINNC')
    prec_n = rainc_n + rainsh_n + rainnc_n

    rainc_np1 = getvar(file_np1,'RAINC')
    rainsh_np1 = getvar(file_np1,'RAINSH')
    rainnc_np1 = getvar(file_np1,'RAINNC')
    prec_np1 = rainc_np1 + rainsh_np1 + rainnc_np1

    prec_hourly = prec_np1 - prec_n

    mean_prec_hourly = (16.0/7.0)*np.mean(prec_hourly)
    sim_hourly_rainfall.append(mean_prec_hourly)

    T = T+1

# Initialize variables to store hourly rainfall and hours
hourly_rainfall = []
hours = np.arange(24)

# Get a list of all files and sort them
file_list = sorted(glob.glob(data_dir1))

# Iterate over the files, processing them in pairs
for i in range(0, len(file_list), 2):
    # Initialize variables to store mean rainfall for the pair
    mean_rainfall = np.zeros((2,))
    
    # Process the current pair of files
    for j, file_path in enumerate(file_list[i:i+2]):
        # Open the file
        ds = xr.open_dataset(file_path)
        
        # Extract precipitation data for the entire half-hour period
        prcp = ds.precipitationCal.sel(lon=slice(lonmin, lonmax), lat=slice(latmin, latmax)).values
        
        # Calculate the mean precipitation over the entire grid
        mean_rainfall[j] = np.mean(prcp)
    
    # Calculate the sum of precipitation for each hour in the pair and append to hourly_rainfall
    hourly_rainfall.append(mean_rainfall.sum())
# Create a plot
plt.figure(figsize=(10, 6))
plt.plot(hours, hourly_rainfall, marker='o', linestyle='-', color='b', markersize=8, label='IMERG')
plt.plot(hours,  sim_hourly_rainfall, marker='o', linestyle='-', color='r', markersize=8, label='WRF')
plt.xlabel('Hour of the Day', fontsize=14)
plt.ylabel('Precipitation (mm)', fontsize=14)
plt.title('Hourly Mean Precipitation', fontsize=16)
plt.xticks(hours)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(fontsize=12)
plt.tight_layout()
#plt.savefig("abcd.png", dpi=300, bbox_inches='tight')
plt.show()